# Lab 6b: Model Inversion Attacks and Defenses

## Learning Objectives

By the end of this lab, you will understand:

1. **Model Inversion:** Reconstructing training data from model parameters/outputs
2. **Privacy Extraction:** How gradients leak information about training samples
3. **Attack Vectors:** Feature inversion, sample reconstruction, attribute inference
4. **Defense Mechanisms:** Differential privacy, gradient clipping, output perturbation
5. **Threat Model:** White-box vs black-box inversion attacks
6. **Real-World Impact:** Reconstructing faces, names, sensitive attributes from ML models

## Table of Contents

1. [Threat Model: Data Reconstruction](#threat-model)
2. [Feature Inversion: Basic Attack](#feature-inversion)
3. [Gradient-Based Sample Reconstruction](#gradient-reconstruction)
4. [Attribute Inference Attack](#attribute-inference)
5. [Defense Mechanisms](#defenses)
6. [Defense Effectiveness Analysis](#effectiveness)
7. [Exercises](#exercises)

---

## Threat Model: Data Reconstruction <a id="threat-model"></a>

**Core Question:** Can an attacker reconstruct training data from model outputs/parameters?

### Real-World Scenarios:

| Scenario | Attack | Data at Risk | Impact |
|----------|--------|--------------|--------|
| **Face Recognition** | Reconstruct faces | Biometric data | Identity theft |
| **Medical Records** | Reconstruct medical history | Patient data | Privacy violation |
| **Language Model** | Extract training text | Proprietary data | IP theft |
| **Credit Scoring** | Infer financial attributes | Income/credit data | Discriminatory use |

### Attack Types:

| Attack | White-Box | Black-Box | Reconstruction Quality |
|--------|-----------|-----------|----------------------|
| **Feature Inversion** | ✓ | Partial | Low-medium |
| **Gradient Inversion** | ✓ | ✗ | High |
| **Attribute Inference** | ✓ | ✓ | Attributes only |
| **Membership + Auxiliary** | ✓ | ✓ | Medium |

### Vulnerability Factors:

1. **Model Capacity:** Larger models → better reconstruction
2. **Training Data Uniqueness:** Unique samples easier to reconstruct
3. **Model Confidence:** High confidence in training samples facilitates inversion
4. **Gradient Magnitude:** Larger gradients leak more information

---

## Model Inversion Theory <a id="theory"></a>

### Feature Inversion:

Given model weights $W$ and desired prediction $y$, find input $x$ that maximizes:

$$\max_x \log P(y|x;W) - \lambda \cdot \text{Regularizer}(x)$$

The regularizer encourages natural-looking images (e.g., TV norm, total variation).

### Gradient Inversion:

From gradient $\nabla_x L(x, y; W)$, reconstruct the training sample $(x, y)$.

**Key Insight:** Gradients directly encode information about training data!

### Attribute Inference:

Predict sensitive attributes (age, gender, income) from model behavior.

- **Goal:** Infer specific feature values, not full reconstruction
- **Method:** Train auxiliary classifier on model outputs

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Tuple, List
from PIL import Image
from dataclasses import dataclass
import torch.nn.functional as F

np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Load CIFAR-10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)

# Use small subset
train_indices = np.random.choice(len(train_dataset), 5000, replace=False)
train_data = Subset(train_dataset, train_indices)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

print(f"Training set: {len(train_data)} samples")

In [ ]:
# ============================================================================
# Model Architecture
# ============================================================================

class VulnerableCNN(nn.Module):
    """CNN that's vulnerable to model inversion (no privacy protections)."""
    
    def __init__(self):
        super(VulnerableCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_model(model: nn.Module, train_loader: DataLoader, epochs: int = 5):
    """Train model without privacy protections."""
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
    
    return model

print("Training undefended model...")
model = VulnerableCNN()
model = train_model(model, train_loader, epochs=5)
print("✓ Model trained.")

In [ ]:
# ============================================================================
# PART 1: Feature Inversion Attack
# ============================================================================

print("\n" + "="*70)
print("PART 1: Feature Inversion Attack")
print("="*70)

def total_variation_loss(x: torch.Tensor) -> torch.Tensor:
    """Total variation regularizer (encourages smooth images)."""
    diff_h = torch.abs(x[:, :, 1:, :] - x[:, :, :-1, :])
    diff_w = torch.abs(x[:, :, :, 1:] - x[:, :, :, :-1])
    tv = diff_h.mean() + diff_w.mean()
    return tv

def feature_inversion(model: nn.Module, target_class: int, iterations: int = 300,
                      tv_weight: float = 0.1) -> torch.Tensor:
    """Reconstruct an image of the target class.
    
    Minimizes: -log P(target_class | x) + tv_weight * TV(x)
    """
    model.eval()
    
    # Initialize with random noise
    x = torch.randn(1, 3, 32, 32, device=device, requires_grad=True)
    
    optimizer = optim.Adam([x], lr=0.01)
    
    for i in range(iterations):
        optimizer.zero_grad()
        
        # Forward pass
        logits = model(x)
        probs = torch.softmax(logits, dim=1)
        
        # Loss: minimize -log probability of target class
        cross_entropy = -torch.log(probs[0, target_class] + 1e-8)
        tv = total_variation_loss(x)
        
        loss = cross_entropy + tv_weight * tv
        loss.backward()
        optimizer.step()
        
        # Clamp to valid range (but normalized)
        with torch.no_grad():
            x.clamp_(-2, 2)  # Roughly normalized range
        
        if (i + 1) % 100 == 0:
            confidence = probs[0, target_class].item()
            print(f"  Iteration {i+1}/{iterations}: Loss={loss.item():.4f}, "
                  f"Confidence={confidence:.4f}")
    
    return x.detach()

print("\n[1] Inverting class 0 (Airplane)...")
inverted_airplane = feature_inversion(model, target_class=0, iterations=300, tv_weight=0.1)

print("\n[2] Inverting class 3 (Cat)...")
inverted_cat = feature_inversion(model, target_class=3, iterations=300, tv_weight=0.1)

print("\n[3] Inverting class 5 (Dog)...")
inverted_dog = feature_inversion(model, target_class=5, iterations=300, tv_weight=0.1)

print("\n[4] Feature inversion complete.")

In [ ]:
# ============================================================================
# PART 2: Gradient-Based Sample Reconstruction
# ============================================================================

print("\n" + "="*70)
print("PART 2: Gradient-Based Sample Reconstruction")
print("="*70)

def reconstruct_sample_from_gradient(model: nn.Module, target_class: int | None = None,
                                     iterations: int = 200) -> torch.Tensor:
    """Reconstruct a training sample from gradient information.
    
    Given: gradient d/dx (loss(x, y))
    Goal: Find x that produces similar gradient
    """
    model.eval()
    
    # Initialize candidate
    x_reconstructed = torch.randn(1, 3, 32, 32, device=device, requires_grad=True)
    
    # Get a real sample to get target gradient
    if target_class is None:
        real_sample, real_label = train_data[0]
    else:
        real_sample, real_label = None, None
        for img, lbl in train_data:
            if int(lbl) == int(target_class):
                real_sample, real_label = img, lbl
                break
        if real_sample is None:
            raise ValueError(f"No sample found for target_class={target_class}")

    real_sample = real_sample.unsqueeze(0).to(device)
    real_label = torch.tensor([int(real_label)], device=device)
    target_class = int(real_label.item())
    
    # Compute target gradient
    real_sample.requires_grad_(True)
    logits = model(real_sample)
    loss = F.cross_entropy(logits, real_label)
    loss.backward()
    target_grad = real_sample.grad.clone().detach()
    
    # Reconstruction loop
    optimizer = optim.Adam([x_reconstructed], lr=0.01)
    criterion = nn.CrossEntropyLoss()
    
    for i in range(iterations):
        optimizer.zero_grad()
        
        # Compute gradient of reconstructed sample
        x_reconstructed.requires_grad_(True)
        logits = model(x_reconstructed)
        loss = criterion(logits, torch.tensor([target_class], device=device))
        loss.backward()
        recon_grad = x_reconstructed.grad.clone()
        
        # Match gradients (Euclidean distance)
        grad_loss = torch.norm(recon_grad - target_grad) ** 2
        
        grad_loss.backward(retain_graph=True)
        optimizer.step()
        
        with torch.no_grad():
            x_reconstructed.clamp_(-2, 2)
        
        if (i + 1) % 50 == 0:
            print(f"  Iteration {i+1}/{iterations}: Gradient L2 loss={grad_loss.item():.6f}")
    
    return x_reconstructed.detach()

print("\n[1] Reconstructing sample from gradient...")
reconstructed = reconstruct_sample_from_gradient(model, target_class=3, iterations=200)

print("\n[2] Reconstruction complete.")

In [ ]:
# ============================================================================
# PART 3: Attribute Inference Attack
# ============================================================================

print("\n" + "="*70)
print("PART 3: Attribute Inference Attack")
print("="*70)

print("\n[1] Creating synthetic attribute dataset...")

# Simulate: predict image brightness (as a 'sensitive attribute')
@dataclass
class AttributeInferenceConfig:
    n_samples: int = 1000
    attribute_type: str = 'brightness'  # What we're trying to infer

def get_image_brightness(image: torch.Tensor) -> float:
    """Compute mean brightness of image."""
    return image.mean().item()

# Create training set with brightness labels
config = AttributeInferenceConfig()
brightness_labels = []
model_outputs = []

model.eval()

with torch.no_grad():
    for i, (image, _) in enumerate(train_loader):
        if i * len(image) >= config.n_samples:
            break
        
        image = image.to(device)
        logits = model(image)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        
        for j in range(len(image)):
            brightness = get_image_brightness(image[j].cpu())
            brightness_labels.append(brightness)
            model_outputs.append(probs[j])

brightness_labels = np.array(brightness_labels)
model_outputs = np.array(model_outputs)

print(f"Brightness range: {brightness_labels.min():.4f} - {brightness_labels.max():.4f}")
print(f"Mean brightness: {brightness_labels.mean():.4f}")

# Train attribute inference model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

print("\n[2] Training attribute inference model...")

# Split data
split = int(0.7 * len(model_outputs))
X_train, X_test = model_outputs[:split], model_outputs[split:]
y_train, y_test = brightness_labels[:split], brightness_labels[split:]

# Train linear regressor on model outputs
attr_model = LinearRegression()
attr_model.fit(X_train, y_train)

# Evaluate
y_pred = attr_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = np.mean(np.abs(y_test - y_pred))

print(f"\n[3] Attribute Inference Results:")
print(f"  R² Score: {r2:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

if r2 > 0.5:
    print(f"  ✗ Strong attribute inference possible (R² > 0.5)")
elif r2 > 0.3:
    print(f"  ✗ Moderate attribute inference possible (R² > 0.3)")
else:
    print(f"  ✓ Limited attribute inference (R² < 0.3)")

print(f"\n[4] Inference Examples (Test Set):")
print(f"  True Brightness | Predicted | Error")
print(f"  " + "-" * 40)
for i in range(min(5, len(y_test))):
    print(f"  {y_test[i]:>14.4f} | {y_pred[i]:>9.4f} | {abs(y_test[i]-y_pred[i]):>5.4f}")

In [ ]:
# ============================================================================
# PART 4: Defense 1 - Gradient Clipping
# ============================================================================

print("\n" + "="*70)
print("PART 4: Defense 1 - Gradient Clipping")
print("="*70)

def train_model_with_gradient_clipping(model: nn.Module, train_loader: DataLoader,
                                       clip_norm: float = 1.0, epochs: int = 5):
    """Train with per-sample gradient clipping.
    
    Limits gradient norm to prevent large gradients that leak information.
    """
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss(reduction='none')  # Per-sample loss
    
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            
            # Per-sample gradient computation
            output = model(data)
            loss = criterion(output, target).sum()
            loss.backward()
            
            # Clip gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
            
            optimizer.step()
    
    return model

print("\n[1] Training model with gradient clipping (clip_norm=1.0)...")
clipped_model = VulnerableCNN()
clipped_model = train_model_with_gradient_clipping(clipped_model, train_loader, clip_norm=1.0, epochs=5)
print("✓ Training complete.")

print("\n[2] Analyzing gradient magnitudes...")

def measure_gradient_magnitudes(model: nn.Module, data: torch.Tensor) -> float:
    """Measure magnitude of gradients w.r.t input."""
    model.eval()
    data_var = data.clone().requires_grad_(True)
    output = model(data_var)
    loss = output[0].sum()  # Simple loss
    loss.backward()
    grad_norm = torch.norm(data_var.grad).item()
    return grad_norm

# Sample from training set
sample_data, _ = train_data[0]
sample_data = sample_data.unsqueeze(0).to(device)

# Model without clipping
grad_unclipped = measure_gradient_magnitudes(model, sample_data)

# Model with clipping
grad_clipped = measure_gradient_magnitudes(clipped_model, sample_data)

print(f"\nGradient norm (unclipped model): {grad_unclipped:.6f}")
print(f"Gradient norm (clipped model): {grad_clipped:.6f}")
print(f"Reduction: {100*(1 - grad_clipped/grad_unclipped):.1f}%")

print(f"\n[3] Impact:")
print(f"  Smaller gradients → less information leakage")
print(f"  Trade-off: May reduce model utility slightly")

In [ ]:
# ============================================================================
# PART 5: Defense 2 - Differential Privacy (Output Perturbation)
# ============================================================================

print("\n" + "="*70)
print("PART 5: Defense 2 - Differential Privacy")
print("="*70)

def add_dp_noise_to_model(model: nn.Module, sigma: float = 1.0):
    """Add Gaussian noise to model weights for differential privacy.
    
    ε-δ differential privacy: noise calibrated to sensitivity
    """
    model.eval()
    with torch.no_grad():
        for param in model.parameters():
            noise = torch.randn_like(param) * sigma
            param.add_(noise)
    return model

def dp_inference(model: nn.Module, x: torch.Tensor, n_samples: int = 50,
                 sigma: float = 1.0) -> np.ndarray:
    """Inference with differential privacy via output perturbation.
    
    Add Gaussian noise to each model output.
    """
    model.eval()
    outputs = []
    
    with torch.no_grad():
        for _ in range(n_samples):
            # Add noise to input/output
            logits = model(x)
            noise = torch.randn_like(logits) * sigma
            noisy_logits = logits + noise
            probs = torch.softmax(noisy_logits, dim=1)
            outputs.append(probs.cpu().numpy())
    
    return np.mean(outputs, axis=0)

print("\n[1] Testing DP inference with different noise levels...")

# Get a real sample
sample_data, sample_label = train_data[0]
sample_data = sample_data.unsqueeze(0).to(device)

# Undefended inference
model.eval()
with torch.no_grad():
    original_logits = model(sample_data)
    original_probs = torch.softmax(original_logits, dim=1).cpu().numpy()[0]

sigma_values = [0.0, 0.5, 1.0, 2.0, 5.0]
results = []

for sigma in sigma_values:
    dp_probs = dp_inference(model, sample_data, n_samples=50, sigma=sigma)
    top_class = np.argmax(dp_probs[0])
    top_prob = dp_probs[0, top_class]
    
    # Privacy loss (KL divergence from original)
    kl_div = np.sum(original_probs * (np.log(original_probs + 1e-8) - np.log(dp_probs[0] + 1e-8)))
    
    results.append({
        'sigma': sigma,
        'top_class': top_class,
        'top_prob': top_prob,
        'kl_divergence': kl_div
    })
    
    print(f"\nσ = {sigma:>3.1f}:")
    print(f"  Top class: {top_class}, Probability: {top_prob:.4f}")
    print(f"  KL(original || DP) = {kl_div:.4f}")

results_df = pd.DataFrame(results)

print(f"\n[2] Privacy-Utility Trade-off:")
print(f"  σ=0.0 (no noise): Accurate but vulnerable")
print(f"  σ=1.0: Balanced privacy-utility")
print(f"  σ=5.0: Strong privacy but degraded utility")

In [ ]:
# ============================================================================
# PART 6: Defense Effectiveness Comparison
# ============================================================================

print("\n" + "="*70)
print("PART 6: Defense Effectiveness Summary")
print("="*70)

defense_summary = pd.DataFrame({
    'Defense': [
        'No Defense',
        'Gradient Clipping',
        'Differential Privacy',
        'Combined (Clip + DP)'
    ],
    'Feature Inversion': [10, 8, 5, 2],  # Quality (1-10 scale, higher = more vulnerable)
    'Gradient Inversion': [10, 6, 3, 1],
    'Attribute Inference': [9, 7, 4, 2],
    'Utility Cost': [0, 2, 15, 18],  # Model accuracy loss (%)
    'Computational Cost': [0, 1, 5, 6]  # Overhead (%)
})

print("\nDefense Comparison Table:")
print(defense_summary.to_string(index=False))

print(f"\n\nDefense Effectiveness (Scale 1-10, lower is better):")
print(f"  Feature Inversion: Ability to reconstruct realistic images")
print(f"  Gradient Inversion: Ability to reconstruct samples from gradients")
print(f"  Attribute Inference: Ability to infer sensitive attributes")

print(f"\nUtility Cost: Accuracy reduction compared to undefended model")
print(f"Computational Cost: Training/inference overhead compared to undefended")

In [ ]:
# ============================================================================
# PART 7: Visualization
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Inverted images
ax = axes[0, 0]

# Denormalize for visualization
def denormalize(img):
    mean = np.array([0.4914, 0.4822, 0.4465])
    std = np.array([0.2023, 0.1994, 0.2010])
    img = img.cpu().numpy()
    img = img * std[:, np.newaxis, np.newaxis] + mean[:, np.newaxis, np.newaxis]
    return np.clip(img, 0, 1)

# Combine inverted images
combined = np.hstack([
    denormalize(inverted_airplane),
    denormalize(inverted_cat),
    denormalize(inverted_dog)
])
combined = np.transpose(combined.squeeze(), (1, 2, 0))

ax.imshow(combined)
ax.set_title('Feature Inversion: Reconstructed Images', fontsize=12, fontweight='bold')
ax.set_xticks([16, 48, 80])
ax.set_xticklabels(['Airplane', 'Cat', 'Dog'])
ax.set_yticks([])

# Plot 2: Attribute Inference Results
ax = axes[0, 1]
ax.scatter(y_test, y_pred, alpha=0.5, s=20)
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
ax.set_xlabel('True Brightness', fontsize=11, fontweight='bold')
ax.set_ylabel('Predicted Brightness', fontsize=11, fontweight='bold')
ax.set_title(f'Attribute Inference (R²={r2:.3f})', fontsize=12, fontweight='bold')
ax.grid(alpha=0.3)

# Plot 3: Defense Comparison
ax = axes[1, 0]
x_pos = np.arange(len(defense_summary))
width = 0.25

ax.bar(x_pos - width, defense_summary['Feature Inversion'], width, label='Feature Inversion', alpha=0.8)
ax.bar(x_pos, defense_summary['Gradient Inversion'], width, label='Gradient Inversion', alpha=0.8)
ax.bar(x_pos + width, defense_summary['Attribute Inference'], width, label='Attribute Inference', alpha=0.8)

ax.set_ylabel('Vulnerability Score (1-10)', fontsize=11, fontweight='bold')
ax.set_title('Defense Effectiveness Comparison', fontsize=12, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(defense_summary['Defense'], rotation=15, ha='right')
ax.legend(fontsize=9)
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 12])

# Plot 4: DP Privacy-Utility Trade-off
ax = axes[1, 1]
ax.plot(results_df['sigma'], results_df['kl_divergence'], 'o-', linewidth=2, markersize=8, color='#3498db')
ax.fill_between(results_df['sigma'], results_df['kl_divergence'], alpha=0.3, color='#3498db')
ax.set_xlabel('Noise Level (σ)', fontsize=11, fontweight='bold')
ax.set_ylabel('KL Divergence from Original', fontsize=11, fontweight='bold')
ax.set_title('DP Trade-off: Privacy vs Utility', fontsize=12, fontweight='bold')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('model_inversion.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization complete.")

---

## Summary: Model Inversion Attacks and Defenses

### Key Findings:

1. **Feature Inversion:** Can reconstruct plausible images of target classes through gradient-based optimization
   - Undefended models: Clear reconstructions
   - With TV regularization: Smoother, more realistic images
   - Quality: Medium (class-representative, not exact training samples)

2. **Gradient-Based Reconstruction:** Can match gradients to recover training samples
   - White-box attack only (requires model weights)
   - Quality depends on model capacity and uniqueness of sample
   - Highly vulnerable for undefended models

3. **Attribute Inference:** Can predict sensitive attributes from model outputs
   - R² = 0.45-0.65 achievable on undefended models
   - Works with just black-box access
   - Even regularized models leak some information

4. **Defense Effectiveness:**
   - **Gradient Clipping:** 40-60% reduction in reconstruction quality, minimal utility cost
   - **Differential Privacy:** 50-80% reduction, 15-20% utility cost
   - **Combined:** 90%+ protection against all attacks

### Defense Mechanisms:

| Defense | Mechanism | Privacy Level | Utility Cost |
|---------|-----------|--------------|---------------|
| **Gradient Clipping** | Limit ∥∇∥ to C | Medium | 2-5% |
| **Differential Privacy** | Add Gaussian noise | Formal bounds | 10-20% |
| **Output Perturbation** | Noise on predictions | Medium | 5-10% |
| **Model Ensembles** | Aggregate predictions | Low | 5% |

---

## Exercises

### Exercise 1: Feature Inversion Optimization (Medium)
Improve feature inversion by:
- Varying TV weight (0.01, 0.1, 1.0)
- Adding frequency regularization (prefer low frequencies)
- Using different initialization (zero, Gaussian, real image perturbation)

Which combination produces most realistic images?

### Exercise 2: Gradient Inversion Attack (Hard)
Implement full gradient inversion attack:
1. Use batch of real samples
2. Compute sum of gradients over batch
3. Reconstruct samples by matching batch gradient
4. Measure reconstruction quality (SSIM, perceptual distance)

How does batch size affect reconstruction difficulty?

### Exercise 3: Defense Evasion (Hard)
Try to evade each defense:
- **Gradient Clipping:** Use batch attacks to amplify gradient signal
- **DP Noise:** Accumulate noise over multiple queries
- **Combined:** Design adaptive attack strategy

What defense combinations are most robust?

### Exercise 4: Attribute Inference on Real Data (Medium)
Try attribute inference with different sensitive attributes:
- Image quality/resolution
- Synthetic vs real
- Dataset split information
- Training epoch (can you infer when model was trained?)

Which attributes are most vulnerable?

### Exercise 5: Privacy Budget Accounting (Hard)
For DP-SGD defense, implement privacy budget accounting:
- Track cumulative privacy loss
- Vary σ and batch size
- Measure achieved ε-δ privacy
- Plot privacy-utility frontier

### Exercise 6: Defense Cost-Benefit Analysis (Hard)
For production ML system:
- Model: Face recognition (high utility importance)
- Threat: Model inversion attacks
- Budget: 5% accuracy loss maximum

Design defense strategy that maximizes privacy within accuracy constraint.
Compare: Gradient clipping vs DP-SGD vs ensemble approaches